In [61]:
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime

def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)  # メル周波数ケプストラム係数（Mel-Frequency Cepstrum Coefficients）
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)  # クロマベクトル(Chroma)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)  # メル周波数ケプストラムの平均値
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)  # スペクトル-コントラクト(Spectral Contrast)の平均
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)  #  和音に関する特徴量(tonal centroid features)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir, sub_dirs, file_ext='*.wav', verbose=0):
    features, labels = np.empty((0,193)), np.empty(0)    # 193次元の特徴量配列
    for label, sub_dir in enumerate(sub_dirs):
        files = glob.glob(os.path.join(parent_dir, sub_dir, file_ext))
        for fn in files:
            if (verbose):
                print('loading: ' + fn)
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])   # hstack: 配列を横方向に結合
            features = np.vstack([features,ext_features])
            paths = fn.split('/')
            labels = np.append(labels, paths[len(paths)-1].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):  # ラベルをスカラー形式からベクトル形式に変換
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [62]:
def load_urbansound8k(parent_dir, sub_dirs, verbose=0):
    file_ext = '*.wav'
    features, labels = parse_audio_files(parent_dir, sub_dirs, file_ext, verbose)
    labels = one_hot_encode(labels)
    return features, labels

def load_urbansound8k_train_and_test(parent_dir = 'UrbanSound8K/audio/',
                                     tr_sub_dirs = ['fold1', 'fold2'], 
                                     ts_sub_dirs = ['fold3'],
                                     verbose = 0):
    tr_features, tr_labels = load_urbansound8k(parent_dir, tr_sub_dirs, verbose)
    ts_features, ts_labels = load_urbansound8k(parent_dir, ts_sub_dirs, verbose)
    return tr_features, tr_labels, ts_features, ts_labels

In [73]:
def save_to_npy_files(tr_features, tr_labels, ts_features, ts_labels):
    np.save('tr_features.npy', tr_features)
    np.save('tr_labels.npy', tr_labels)
    np.save('ts_features.npy', tr_features)
    np.save('ts_labels.npy', tr_labels)

In [74]:
def load_from_npy_files():
    tr_features = np.load('tr_features.npy')
    tr_labels = np.load('tr_labels.npy')
    ts_features = np.load('ts_features.npy')
    ts_labels = np.load('ts_labels.npy')
    return tr_features, tr_labels, ts_features, ts_labels

In [ ]:
if __name__ == '__main__':
    parent_dir = 'UrbanSound8K/audio/'
    tr_subdirs = ['fold1','fold2']
    ts_subdirs =['fold3']
    
    print('loading urbansound8k dataset.')    
    %time tr_features, tr_labels, ts_features, ts_labels = load_urbansound8k_train_and_test(parent_dir,  tr_subdirs, ts_subdirs, 1)
    save_to_npy_files(tr_features, tr_labels, ts_features, ts_labels)


loading urbansound8k dataset.
loading: UrbanSound8K/audio/fold1/101415-3-0-2.wav
loading: UrbanSound8K/audio/fold1/101415-3-0-3.wav
loading: UrbanSound8K/audio/fold1/101415-3-0-8.wav
loading: UrbanSound8K/audio/fold1/102106-3-0-0.wav
loading: UrbanSound8K/audio/fold1/102305-6-0-0.wav
loading: UrbanSound8K/audio/fold1/102842-3-0-1.wav
loading: UrbanSound8K/audio/fold1/102842-3-1-0.wav
loading: UrbanSound8K/audio/fold1/102842-3-1-5.wav
loading: UrbanSound8K/audio/fold1/102842-3-1-6.wav
loading: UrbanSound8K/audio/fold1/103074-7-0-0.wav
loading: UrbanSound8K/audio/fold1/103074-7-0-1.wav
loading: UrbanSound8K/audio/fold1/103074-7-0-2.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-0.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-1.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-2.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-3.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-4.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-5.wav
loading: UrbanSound8K/audio/fold1/103074-7-1-6.wav
l

loading: UrbanSound8K/audio/fold1/134717-0-0-4.wav
loading: UrbanSound8K/audio/fold1/134717-0-0-5.wav
loading: UrbanSound8K/audio/fold1/134717-0-0-6.wav
loading: UrbanSound8K/audio/fold1/134717-0-0-7.wav
loading: UrbanSound8K/audio/fold1/134717-0-0-8.wav
loading: UrbanSound8K/audio/fold1/134717-0-0-9.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-32.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-37.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-40.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-49.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-50.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-65.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-85.wav
loading: UrbanSound8K/audio/fold1/135776-2-0-90.wav
loading: UrbanSound8K/audio/fold1/137156-9-0-28.wav
loading: UrbanSound8K/audio/fold1/137156-9-0-30.wav
loading: UrbanSound8K/audio/fold1/137156-9-0-31.wav
loading: UrbanSound8K/audio/fold1/137156-9-0-5.wav
loading: UrbanSound8K/audio/fold1/137156-9-0-73.wav
loading: UrbanSound

loading: UrbanSound8K/audio/fold1/161702-9-0-37.wav
loading: UrbanSound8K/audio/fold1/161702-9-0-45.wav
loading: UrbanSound8K/audio/fold1/161702-9-0-6.wav
loading: UrbanSound8K/audio/fold1/161702-9-0-8.wav
loading: UrbanSound8K/audio/fold1/162540-1-0-0.wav
loading: UrbanSound8K/audio/fold1/162540-1-1-0.wav
loading: UrbanSound8K/audio/fold1/162540-1-2-0.wav
loading: UrbanSound8K/audio/fold1/162703-1-0-0.wav
loading: UrbanSound8K/audio/fold1/164053-8-0-0.wav
loading: UrbanSound8K/audio/fold1/164053-8-1-0.wav
loading: UrbanSound8K/audio/fold1/164053-8-2-0.wav
loading: UrbanSound8K/audio/fold1/164053-8-2-1.wav
loading: UrbanSound8K/audio/fold1/164053-8-2-2.wav
loading: UrbanSound8K/audio/fold1/164053-8-2-3.wav
loading: UrbanSound8K/audio/fold1/164053-8-3-0.wav
loading: UrbanSound8K/audio/fold1/164312-3-0-0.wav
loading: UrbanSound8K/audio/fold1/164312-3-1-0.wav
loading: UrbanSound8K/audio/fold1/164312-3-3-0.wav
loading: UrbanSound8K/audio/fold1/164312-3-4-0.wav
loading: UrbanSound8K/audio/f

loading: UrbanSound8K/audio/fold1/180937-7-1-4.wav
loading: UrbanSound8K/audio/fold1/180937-7-1-5.wav
loading: UrbanSound8K/audio/fold1/180937-7-1-6.wav
loading: UrbanSound8K/audio/fold1/180937-7-1-7.wav
loading: UrbanSound8K/audio/fold1/180937-7-1-8.wav
loading: UrbanSound8K/audio/fold1/180937-7-1-9.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-0.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-1.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-10.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-2.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-3.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-4.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-5.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-6.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-7.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-8.wav
loading: UrbanSound8K/audio/fold1/180937-7-2-9.wav
loading: UrbanSound8K/audio/fold1/180937-7-3-0.wav
loading: UrbanSound8K/audio/fold1/180937-7-3-1.wav
loading: UrbanSound8K/audio/fo

/Users/hashizume/anaconda/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


...

loading: UrbanSound8K/audio/fold3/144068-5-0-11.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-12.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-13.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-14.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-15.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-16.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-17.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-18.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-2.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-3.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-4.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-5.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-6.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-7.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-8.wav
loading: UrbanSound8K/audio/fold3/144068-5-0-9.wav
loading: UrbanSound8K/audio/fold3/144068-5-1-0.wav
loading: UrbanSound8K/audio/fold3/144068-5-2-0.wav
loading: UrbanSound8K/audio/fold3/144068-5-3-0.wav
loading: UrbanSound8K/a

loading: UrbanSound8K/audio/fold3/169044-2-0-21.wav
loading: UrbanSound8K/audio/fold3/169044-2-0-25.wav
loading: UrbanSound8K/audio/fold3/169044-2-0-3.wav
loading: UrbanSound8K/audio/fold3/169044-2-0-5.wav
loading: UrbanSound8K/audio/fold3/170015-3-0-0.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-105.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-113.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-203.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-211.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-212.wav
loading: UrbanSound8K/audio/fold3/172315-9-0-224.wav
loading: UrbanSound8K/audio/fold3/174840-1-0-0.wav
loading: UrbanSound8K/audio/fold3/175847-1-0-0.wav
loading: UrbanSound8K/audio/fold3/175854-1-0-0.wav
loading: UrbanSound8K/audio/fold3/17615-3-0-0.wav
loading: UrbanSound8K/audio/fold3/17615-3-0-3.wav
loading: UrbanSound8K/audio/fold3/17615-3-0-4.wav
loading: UrbanSound8K/audio/fold3/17615-3-0-6.wav
loading: UrbanSound8K/audio/fold3/176631-1-0-0.wav
loading: UrbanSound8K

loading: UrbanSound8K/audio/fold3/195451-5-0-0.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-1.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-10.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-11.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-12.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-13.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-14.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-15.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-16.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-17.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-18.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-19.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-2.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-20.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-21.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-3.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-4.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-5.wav
loading: UrbanSound8K/audio/fold3/195451-5-0-6.wav
loading: UrbanSound

loading: UrbanSound8K/audio/fold3/34708-6-4-0.wav
loading: UrbanSound8K/audio/fold3/34708-6-5-0.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-0.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-1.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-2.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-3.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-4.wav
loading: UrbanSound8K/audio/fold3/37560-4-0-5.wav
loading: UrbanSound8K/audio/fold3/41372-3-0-25.wav
loading: UrbanSound8K/audio/fold3/41372-3-0-36.wav
loading: UrbanSound8K/audio/fold3/41372-3-0-39.wav
loading: UrbanSound8K/audio/fold3/41372-3-0-51.wav
loading: UrbanSound8K/audio/fold3/42097-7-0-0.wav
loading: UrbanSound8K/audio/fold3/42097-7-0-1.wav
loading: UrbanSound8K/audio/fold3/42097-7-0-2.wav
loading: UrbanSound8K/audio/fold3/42117-8-0-0.wav
loading: UrbanSound8K/audio/fold3/42117-8-0-1.wav
loading: UrbanSound8K/audio/fold3/42117-8-0-10.wav
loading: UrbanSound8K/audio/fold3/42117-8-0-11.wav
loading: UrbanSound8K/audio/fold3/42117-8-0-

loading: UrbanSound8K/audio/fold3/63095-4-0-6.wav
loading: UrbanSound8K/audio/fold3/63095-4-0-8.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-10.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-18.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-19.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-20.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-23.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-27.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-33.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-4.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-6.wav
loading: UrbanSound8K/audio/fold3/63095-4-1-7.wav
loading: UrbanSound8K/audio/fold3/63292-3-0-0.wav
loading: UrbanSound8K/audio/fold3/63292-3-0-1.wav
loading: UrbanSound8K/audio/fold3/65750-3-0-5.wav
loading: UrbanSound8K/audio/fold3/65750-3-3-48.wav
loading: UrbanSound8K/audio/fold3/65750-3-3-68.wav
loading: UrbanSound8K/audio/fold3/65750-3-3-74.wav
loading: UrbanSound8K/audio/fold3/66622-4-0-0.wav
loading: UrbanSound8K/audio/fold3/66622-